In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import VisualBertModel, VisualBertConfig, BertTokenizer
from transformers import LxmertModel, LxmertConfig
from sklearn.model_selection import train_test_split
import pickle
import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
!ls "../Nish/VBExperiments/cifar10-train-embeddings.pkl"

Archived
cifar10-train-embeddings.pkl
EmptyString-Cifar10.ipynb
EmptyString.ipynb
EmptyString-TopBox.ipynb
extracting_data.py
get_cifar100_embeddings.ipynb
get_cifar100_embeddings_top_box_only.ipynb
get_cifar10_embeddings.ipynb
modeling_frcnn.py
processing_image.py
__pycache__
Question-AllClassesListed-Cifar10.ipynb
Question-AllClassesListed.ipynb
Question-WhatIsThisImage-Cifar10.ipynb
Question-WhatIsThisImage.ipynb
utils.py
visualizing_image.py
VQA1.ipynb


In [2]:
class Cifar10Dataset(Dataset):
    def __init__(self, visual_embeddings,bounding_boxes, labels, tokenizer):
        self.visual_embeddings = visual_embeddings
        self.bounding_boxes = bounding_boxes
        self.labels = labels
        self.tokenizer = tokenizer
        self.num_labels = len(set(self.labels))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        visual_embedding = self.visual_embeddings[idx]
        bounding_box = self.bounding_boxes[idx]
        # text_inputs = tokenizer(text_inputs, padding="max_length", max_length=32, truncation=True, return_tensors="pt")
        text_inputs = tokenizer("", padding="max_length", max_length=32, truncation=True, return_tensors="pt")
        
        # remove batch dimension
        for k,v in text_inputs.items():
            text_inputs[k] = v.squeeze()
            
        label = torch.zeros(self.num_labels)
        label[self.labels[idx]] = 1
        return visual_embedding, bounding_box, text_inputs, label

In [3]:
visual_bert = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")
print(visual_bert.config)

lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-vqa-uncased")
print(lxmert.config)

Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


VisualBertConfig {
  "_name_or_path": "uclanlp/visualbert-vqa-coco-pre",
  "architectures": [
    "VisualBertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "bypass_transformer": false,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "visual_bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "special_visual_initialize": true,
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "visual_embedding_dim": 2048,
  "vocab_size": 30522
}



Some weights of the model checkpoint at unc-nlp/lxmert-vqa-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.weight', 'answer_head.logit_fc.3.weight', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.0.bias', 'answer_head.logit_fc.3.bias']
- This IS expected if you are initializing LxmertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LxmertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LxmertConfig {
  "_name_or_path": "unc-nlp/lxmert-vqa-uncased",
  "architectures": [
    "LxmertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "l_layers": 9,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "lxmert",
  "num_attention_heads": 12,
  "num_attr_labels": 400,
  "num_hidden_layers": {
    "cross_encoder": 5,
    "language": 9,
    "vision": 5
  },
  "num_object_labels": 1600,
  "num_qa_labels": 3129,
  "r_layers": 5,
  "task_mask_lm": true,
  "task_matched": true,
  "task_obj_predict": true,
  "task_qa": true,
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "visual_attr_loss": true,
  "visual_feat_dim": 2048,
  "visual_feat_loss": true,
  "visual_loss_normalizer": 6.67,
  "visual_obj_loss": true,
  "visual_pos_dim": 4,
  "vocab_size": 30522,
  "x_layers": 5
}



In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [29]:
class LXMERTForImageClassification(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.lxmert = LxmertModel.from_pretrained(model_name)
        self.classifier = nn.Sequential(
            nn.Linear(in_features=self.lxmert.config.hidden_size, out_features=1536, bias=True),
            nn.LayerNorm((1536,), eps=1e-05, elementwise_affine=True),
            nn.GELU(approximate='none'),
            nn.Linear(in_features=1536, out_features=num_classes, bias=True)
        )

    def forward(self, visual_embeddings, bounding_box, text_inputs):         
        outputs = self.lxmert(
            input_ids=text_inputs["input_ids"],
            attention_mask=text_inputs["attention_mask"],
            token_type_ids=text_inputs["token_type_ids"],
            visual_feats=visual_embeddings,
            visual_pos = bounding_box,
            
            visual_attention_mask=torch.ones(visual_embeddings.shape[:-1]).to(device),
        )
        return self.classifier(outputs.pooled_output)

In [30]:
def collate_fn(batch):
        visual_embeddings,bounding_box, text_inputs, labels = zip(*batch)
        
        # Flatten the visual embeddings
        # visual_embeddings = [embedding.view(embedding.size(0), -1) for embedding in visual_embeddings]
        visual_embeddings = [embedding.view(embedding.size(1), -1) for embedding in visual_embeddings]
        
        visual_embeddings = torch.stack(visual_embeddings)
        
        bounding_box = [box.view(box.size(1), -1) for box in bounding_box]
        
        bounding_box = torch.stack(bounding_box)
        
        labels = torch.stack(labels)
        
        input_ids = torch.stack([item['input_ids'] for item in text_inputs])
        attention_mask = torch.stack([item['attention_mask'] for item in text_inputs])
        token_type_ids = torch.stack([item['token_type_ids'] for item in text_inputs])
        
        text_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids
        }
        
                
        return visual_embeddings,bounding_box, text_inputs, labels

In [31]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for visual_embeddings, text_inputs, labels in dataloader:
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(visual_embeddings, text_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

In [21]:
with open('cifar10-train-embeddings.pkl', 'rb') as f:
    train_embeddings_all = pickle.load(f)

with open('cifar10-test-embeddings.pkl', 'rb') as f:
    test_embeddings_all = pickle.load(f)

In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_embeddings = train_embeddings_all['embeddings']
train_bounding_box = train_embeddings_all['bounding_box']
train_labels = train_embeddings_all['labels']

val_embeddings = test_embeddings_all['embeddings']
val_bounding_box = test_embeddings_all['bounding_box']
val_labels = test_embeddings_all['labels']
num_classes = len(set(train_labels))

In [33]:
print(train_embeddings[0].shape)
print(train_embeddings[635].shape)
print(train_embeddings[9121].shape)
print(train_bounding_box[0].shape)
print(train_bounding_box[635].shape)
print(train_bounding_box[9121].shape)

torch.Size([1, 36, 2048])
torch.Size([1, 36, 2048])
torch.Size([1, 36, 2048])
torch.Size([1, 36, 4])
torch.Size([1, 36, 4])
torch.Size([1, 36, 4])


In [34]:
model = LXMERTForImageClassification("unc-nlp/lxmert-vqa-uncased", num_classes=num_classes)
model = model.to(device)

Some weights of the model checkpoint at unc-nlp/lxmert-vqa-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.weight', 'answer_head.logit_fc.3.weight', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.0.bias', 'answer_head.logit_fc.3.bias']
- This IS expected if you are initializing LxmertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LxmertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
model

LXMERTForImageClassification(
  (lxmert): LxmertModel(
    (embeddings): LxmertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LxmertEncoder(
      (visn_fc): LxmertVisualFeatureEncoder(
        (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
        (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (box_fc): Linear(in_features=4, out_features=768, bias=True)
        (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer): ModuleList(
        (0-8): 9 x LxmertLayer(
          (attention): LxmertSelfAttentionLayer(
            (self): LxmertAttention(
             

In [36]:
train_dataset = Cifar10Dataset(train_embeddings, train_bounding_box,train_labels, tokenizer)
val_dataset = Cifar10Dataset(val_embeddings, val_bounding_box, val_labels, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)

In [37]:
for name, param in model.named_parameters():
    if 'classifier' in name or 'pooler' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=6e-4)
num_epochs = 30

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.25)

best_params = None
best_val_accuracy = -1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    train_predictions = []
    train_labels_eval = []
    
    for visual_embeddings, bounding_boxes, text_inputs, labels in tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        visual_embeddings = visual_embeddings.to(device)
        bounding_boxes = bounding_boxes.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        
        # import ipdb; ipdb.set_trace()
        outputs = model(visual_embeddings, bounding_boxes, text_inputs)
        _, preds = torch.max(outputs, 1)
        
        train_predictions.extend(preds.cpu().numpy())
        train_labels_eval.extend(labels.cpu().numpy())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    scheduler.step()
    
    train_loss = total_loss / len(train_dataloader)
    
    # Evaluate the model on the validation set
    model.eval()
    val_predictions = []
    val_labels_eval = []
        
    with torch.no_grad():
        for visual_embeddings, bounding_boxes, text_inputs, labels in val_dataloader:
            visual_embeddings = visual_embeddings.to(device)
            bounding_boxes = bounding_boxes.to(device)
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
            labels = labels.to(device)
            
            outputs = model(visual_embeddings,bounding_boxes, text_inputs)
            _, preds = torch.max(outputs, 1)
            
            val_predictions.extend(preds.cpu().numpy())
            val_labels_eval.extend(labels.cpu().numpy())
    
    val_labels_idx = [np.argmax(tensor) for tensor in val_labels_eval]
    val_accuracy = accuracy_score(val_labels_idx, val_predictions)
    
    train_labels_idx = [np.argmax(tensor) for tensor in train_labels_eval]
    train_accuracy = accuracy_score(train_labels_idx, train_predictions)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Training Acc: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:46<00:00,  1.18it/s]


Epoch 1/30, Loss: 1.3138, Training Acc: 0.5298, Validation Accuracy: 0.5837


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:43<00:00,  1.20it/s]


Epoch 2/30, Loss: 1.1417, Training Acc: 0.5912, Validation Accuracy: 0.6130


Epoch 3/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:55<00:00,  1.11it/s]


...

Epoch 28/30, Loss: 0.7922, Training Acc: 0.7167, Validation Accuracy: 0.7051


Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [03:24<00:00,  1.04s/it]


Epoch 29/30, Loss: 0.7879, Training Acc: 0.7184, Validation Accuracy: 0.7136


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [03:25<00:00,  1.05s/it]


Epoch 30/30, Loss: 0.7896, Training Acc: 0.7174, Validation Accuracy: 0.7050
